In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
%matplotlib inline

In [86]:
def data_transform(df):
    df = df.drop(['Unnamed: 0', 'Тип топлива'], axis=1)
    for col in df.columns:
        if df[col].dtypes == object:
            df[col] = df[col].astype('category')


    new_names_col = ['brend', 'model', 'year', 'engine',
           'hp', 'transmisson', 'seats',
           'door', 'mileage', 'price']


    dictionary = dict(zip(list(df.columns), new_names_col))
    df = df.rename(columns=dictionary)

    df.loc[df['year'] < 1994.0, ['year']] = None
    df.loc[df['engine'] > 6500.0, ['engine']] = None
    df.loc[df['engine'] < 660.0, ['engine']] = None
    df.loc[df['hp'] < 40.0, ['hp']] = None
    df.loc[df['hp'] > 500.0, ['hp']].loc[df['brend']=='volvo'] = 240
    df.loc[df['seats'] < 2, ['seats']] = None
    df.loc[df['seats'] > 9, ['seats']] = None
    df.loc[df['door'] < 2 , ['door']] = None
    df.loc[df['door'] > 6  , ['door']] = None

    df['seats'] = df['seats'].fillna(df['seats'].median())
    df['door'] = df['door'].fillna(df['door'].median())
    df['year'] = df['year'].fillna(df['year'].median())
    df['hp'] = df['hp'].fillna(df['hp'].median())
    df['engine'] = round((df['engine']//1)/1000, 1)
    df['year'] = 2015 - df['year']
    
    return df

In [97]:
def get_engine(df):
    X_test_engine = df.loc[df['engine'].isnull(), ['brend', 'model', 'year',
       'hp', 'transmisson', 'seats',
       'door', 'mileage', 'price']]

    X_train_engine = df.loc[df['engine'].isnull() == False, ['brend', 'model', 'year',
       'hp', 'transmisson', 'seats',
       'door', 'mileage', 'price']]

    y_train_engine = df.loc[df['engine'].isnull() == False, ['engine']]
    
    model_LGBM_FF = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', num_leaves=50,
                                learning_rate=0.5, n_estimators=8, max_depth=8)
    
    model_LGBM_FF.fit(X_train_engine, y_train_engine)
    pred_engine = model_LGBM_FF.predict(X_test_engine)

    pred_engine = pd.Series(pred_engine,index=df.loc[df['engine'].isnull(),'engine'].index)
    pred_engine = round(pred_engine, 1)
    
    return pred_engine

In [108]:
def get_model(df):
    X_test = df.loc[df['model'].isnull(), ['brend', 'engine', 'year',
       'hp', 'transmisson', 'seats',
       'door', 'mileage', 'price']]
    X_train = df.loc[df['model'].isnull() == False, ['brend', 'engine', 'year',
       'hp', 'transmisson', 'seats',
       'door', 'mileage', 'price']]
    y_train = df.loc[df['model'].isnull() == False, ['model']]
    
    KNN = KNeighborsClassifier(n_neighbors=3)
    KNN.fit(X_train, y_train)
    
    
    return KNN.predict(X_test)

In [87]:
df = pd.read_csv('data_train.csv')

In [88]:
df = data_transform(df)

In [98]:
df['engine'].fillna(get_engine(df), inplace=True)

In [110]:
df.head()

,brend,model,year,engine,hp,transmisson,seats,door,mileage,price
0,audi,a8,15.0,3.3,166.0,авто,5.0,4.0,352640.0,101800
1,nissan,juke,2.0,1.6,81.0,ручной,5.0,4.0,63460.0,250200
2,skoda,fabia,1.0,1.2,55.0,ручной,5.0,5.0,15865.0,201700
3,audi,a3,2.0,2.0,110.0,ручной,5.0,4.0,44606.3,491300
4,bmw,NaN,16.0,2.8,142.0,ручной,5.0,4.0,187150.0,56100


In [111]:
model = get_model(df)

ValueError: could not convert string to float: 'audi'